# Tech Talk: Exploratory data analysis & interactive figures with Plotly

In this tutorial we will be trying to learn as much as we can from a dataset about which we might not know anything before hand. We will be using Exploratory Data Analysis ([EDA](https://en.wikipedia.org/wiki/Exploratory_data_analysis)) which heavily relies on producing **summary statistics** from the data along with **visual representation**. We use [`plotly`](https://plot.ly/) as our visualisation tool and [`pandas`](http://pandas.pydata.org/) to quickly manipulate data. Together they form a great toolkit to build interacive graphs step-by-step.

![The Anatomy of Beer](./anatomy_of_beer.png)

The theme of this year's [Cambridge Beer Festival (43rd)](http://www.cambridgebeerfestival.com/cbf) is to celebrate the 250th anniversary of the opening of Addenbrooke's Hospital in 1766.

## Loading the Libraries

We start off by importing the modules that are required for the tutorial.

They include:

* `pandas` - the data processing library
* `plotly` - the plotting library

In [ ]:
import pandas as pd
import plotly as py

If there is no error message you should be all setup and ready to go.

If you get an error, it is possible that the libraries are not yet installed. You can install them using `pip` or if you installed python with anaconda you can use `conda`. Go back to your terminal and type:

    pip install plotly
    pip install pandas

OR for anaconda installations:

    conda install plotly
    conda install pandas
    
Re-execute the previous cell and you should now be ready to go.

## Getting the dataset

Along with this notebook you will have downloaded a dataset called `dataset.csv` (I compiled it from two tables found [here](https://github.com/larsga/py-snippets/tree/master/machine-learning/ratebeer), along with some location data). We now need to load it into python using the `pandas` library with the following:

In [ ]:
dataset = pd.read_csv('./dataset.csv')  # this points to the location of the dataset to be loaded

To get an impression of the dataset we can quickly check how large it is in terms of number of columns and rows. The shape attribute is very convenient for this.

In [ ]:
dataset.shape  # .shape returns the dimensions of the table

We have 21 columns and 5147 rows. This is good to know so we can already plan the types of visualisations that could cope with these amounts of data, or whether the data should be aggregated or summarised in some way to facilitate visualisation.

Another way to have a peak is to show the head of the table with `head()`. This is very useful as we can immediately see what the columns and rows correspond to.

In [ ]:
dataset.head()

In this case, every row represents a beer and the different columns are features describing the beers. Some columns are numerical, such as e.g. `Aroma` and `Appearance` which are described by integers and `MyScore` which is in decimal numbers. Some other columns seem to be classes, such as `Style` or locations (e.g. `Country` and `State`).

It is important to learn what data we can plot directly and what data would probably require more preprocessing. For example the data in the `Comments` is not very useful.

To further refine our initial impression we can use the `.describe()` method to get useful summary statistics on a per-column basis. The statistics are indicated neatly as rownames. (Note: this table could be considered as our first plot in that it visually displays the data)

In [ ]:
dataset.describe()

Quite conveniently the `describe()` method only selects the numerical columns to calculate the statistics. Some of the column-wise statistics are more useful than others. For instance we can now say that the `Overall` score ranges from 1 to 19 with an average of 12.27 across all beers. The `BeerID` column is of course completely useless here.

Depending on what we want to do with the dataset we have now many choices. Given the different scores for the beers, it would be interesting to try to understand what makes a good and a bad beer. Here I will look into trying to **visually** distinguish the different beers based on the different columns that describe them.

## Starting the Visual Exploration

Before we get into exploring the depths of the dataset, we will first have a step-by-step guide on how to build a Plotly graph.

![Mapping Data to Aesthetics](./map_data_to_aesthetic.png)

Plotly works with traces. Traces are dictionaries that contain both the data of a plot, as well as instructions on how to format it. Let’s generate one of these traces to examine what they are made out of. First, we load the plotly library and from within it some of the plots types that we want to generate, i.e. Scatter for scatter plots and Box for boxplots. (Note: the `py.offline.init_notebook_mode()` line is required to generate plots when not accessing their API with your registered credentials).

In [ ]:
import plotly.graph_objs as go

py.offline.init_notebook_mode() # run at the start of every ipython notebook so that we can work offline

First we pick two dimensions of the data that we are interested in:

In [ ]:
feature_1 = "Sweet"  # we store the column name of the feature we want to use in the variable feature_1
feature_2 = "Overall"  # the same is done for feature_2 as we can then always reuse them conveniently

Then we choose how we want to visually represent this data. In this case we will map the first feature to the x-axis and the second feature to the y-axis. We want each observation to be marked as a point (or marker in the Plotly lingo).

In [ ]:
scatter_plot_trace = go.Scatter(
    x = dataset[feature_1],  # from the dataset we choose the column that is named "Sweet"
    y = dataset[feature_2],
    mode = "markers",  # we want the observations to be points
    marker = dict(  # this object describes what the points will look like
        size = 4  # this is the size of the points; we could also change the colors
    )
)

This does not do any plotting yet, it just stores the trace as an object that we will later use to produce the actual plot. Let us first have a quick look at what this trace is:

In [ ]:
print(scatter_plot_trace)

For example, this scatterplot contains a series of items including 'type' that is set to 'scatter' and 'mode' that is set to 'markers' to tell plotly to do a scatter plot with points (or markers - as opposed to lines). Then along with the 'markers' mode, it contains an item 'marker' which itself is a dictionary that contains the instructions of how to style the individual markers; here they are set to 'size = 4'. Finally, the trace contains the data in the 'x' and 'y' items, which together define the coordinates of the points. Thus, each trace is a self-contained plotting unit composed of both the formatting information and the data itself.

To plot the trace we create a figure dictionary and assign it to the 'data' item. The figure can then be visualised.

In [ ]:
traces = [scatter_plot_trace]  # the traces that make up our plot have to be stored in a list, even there is only one

fig = go.Figure(data = traces)  # we build a figure object that takes the traces as an input

py.offline.iplot(fig)  # finally we plot the figure object

This gives us our first Plotly plot. Although it is very bare-bones, it already comes with interactivity; when hovering over the individual points, it highlights the coordinates in tooltips. However, we want to do better than this. For instance, we would like at the very least to add the axis titles.

To achieve this, we need to build a layout dictionary and call it to see what it contains:

In [ ]:
scatter_plot_layout = go.Layout(
    xaxis = dict(title = feature_1),  # we reuse the names
    yaxis = dict(title = feature_2)
)

print(scatter_plot_layout)

As we can see, this builds just another dictionary with items that are informative, in this case the feature names that we defined above. To display them on the plot, we have to modify the figure dictionary and add a layout item; note that it is not a list. Then we can replot.

In [ ]:
fig = go.Figure(data = traces, layout = scatter_plot_layout)  # combine the data with the layout into a figure object

py.offline.iplot(fig)

This is already better. However, this plot is not very informative in its current form. There might be many overlapping points. It would be good to get an impression of the number of beers across the different regions of the plot. To achieve this we will be using another *geometry*, `Histogram2d`. The heatmap will count the number of individual beers that fall into each possible combination of values in `Sweet`ness and `Overall` score and assign this number a color. (Note: We can reuse the layout from the previous plot as it will be the same)

In [ ]:
histogram2d_plot_trace = go.Histogram2d(
    x = dataset[feature_1],  # we map the data of feature_1 to the x aesthetic of the geometry
    y = dataset[feature_2]
)

traces = [histogram2d_plot_trace]  # again the single trace needs to be in a list

fig = go.Figure(data = traces, layout = scatter_plot_layout)

py.offline.iplot(fig)

This is much more informative. We see that the bulk of the data was actually located at around `Sweet`ness 0.6 and `Overall` score of 13. Perhaps it also looks like sweeter beers have higher overall ratings. The interactivity of `plotly` makes it very easy to lookup individual values in the heatmap.

There are many more features in plotly that you can explore; for instance we could also stack plots. In the following sections I will show some more examples of how to use `plotly` in combination with simple data analysis.

## Use Case: What countries produce the best beers?

We have had a brief look at the general layout of the data earlier in the tutorial. Now we are going to calculate some summary statistics to gain better insights into the data.

### Group by Country

To get an impression of the Overall scores for the beers in the different countries we would ideally want to calculate the average scores on a per country basis. In pandas this can be achieved with the `.groupby()` method.


In [ ]:
dataset_country = dataset['Overall'].groupby(dataset['Country'])

dataset_country

This gives us a grouped form of the dataset that is not of much use yet. From this dataset we will first have to calculate summary statistics. Some of the statistics that we perhaps would like to know are the minimum score, the maximum score, the number of beers that were scored and the mean score they received - crucially this time - for each country.

Below we define a function that will calculate these different statistics.

In [ ]:
def get_stats(group):
    '''function that calculates the min, max, number of rows and mean value per group.'''
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

Then we `apply()` this function to the grouped dataset. This means that it takes all the beers for one country at a time and *applies* the summary statistics calculations.

![Groupby to Calculate Summary Statistics](./group_data_and_summarize.png)

In [ ]:
stats_country = dataset_country.apply(get_stats).unstack()  # the unstack is required to retain a convenient table

stats_country.head()

This leaves us with a table with one row per country and with 4 columns for each descriptive statistic.

For more help and examples visit Chris Albon's Blog which I find very useful. Specifically for the `.groupby()` operation go [here](http://chrisalbon.com/python/pandas_apply_operations_to_groups.html) and for accessing subsets of the dataset by indexing or filtering go [here](http://chrisalbon.com/python/pandas_index_select_and_filter.html). For a more complete set of instructions go to the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/groupby.html).

### Plot Country Scatter

Now that we have a dataset, we could just be looking up the values in the table or use them to build a plot. Let us build a `Scatter` plot where each point is a country relating the number of beers rated in that country vs. the average rating of these beers.

In [ ]:
stats_country_trace = go.Scatter(  # we picked the Scatter plot geometry
    x = stats_country["mean"],  # we map the mean to the x aesthetic
    y = stats_country["count"],
    mode = "markers",
    marker = dict(
        size = 16,  # the points are quite large with size 16
        color = stats_country["count"]  # we map the number of beers to the color aesthetic
    ),
    text = stats_country.index.tolist()  # we want the tooltips to say what Country we point at
)

traces = [stats_country_trace]  # remember that the traces need to be in a list

layout = go.Layout(
    title = "Beer Countries",  # the title of the plot
    xaxis = dict(title = "Mean Overall Score"),
    yaxis = dict(title = "Number of Beers")
)

fig = go.Figure(data = traces, layout = layout)

py.offline.iplot(fig)


From this plot we can see that generally speaking the countries that have had many different beers have beers that are on average rated higher. We can also see that there are some countries with very high numbers of beers rate.

### Plot on Globe

In the previous plot, while hovering over the countries, we could see that many Scandinavian countries and other Western European countries were among those countries with the most beers rated. To see where in the world the most beers are located, we can plot it on a geographical map.

To do this we first need to extract the longitudes and latitudes of countries. We can do this from the original dataset which contains all that information. We perform a couple of tricks to get there; first we select the column for which we want to remove duplicates (i.e. `Country`) and then extract the data that we want (i.e. `Country`, `longitude` and `latitude`). Finally the `.set_index()` is used to merge in a later step.

In [ ]:
countries = dataset.drop_duplicates(['Country'])[['Country','longitude','latitude']].set_index('Country')

countries.head()

We then merge this information with the `merge()` function. This function matches up two tables by a common column, in this case Country.

In [ ]:
mapped_stats_country = pd.merge(
    stats_country,
    countries,
    right_index = True,
    left_index = True
)

mapped_stats_country.head()

We end up with one table that contains all the information, i.e. the statistics and the location per country.

Before we proceed to plotting this data we need to attend to a litte detail. In the plot below we will map the number of beers rated per country to the size aesthetic of the points. The function below will help to visualise all the points so that none will be too large or too small.

In [ ]:
def rescale_to_range(a, b, x):
    '''
    rescale values to be within a specified range
        a - minimum value
        b - maximum value
        x - data in column to be rescaled
    http://stackoverflow.com/questions/5294955/how-to-scale-down-a-range-of-numbers-with-a-known-min-and-max-value
    '''
    return((((b - a) * (x - x.min())) / (x.max() - x.min())) + a)

mapped_stats_country['scaledCount'] = rescale_to_range(a=10, b=50, x=mapped_stats_country['count'])    

mapped_stats_country.head()

We have scaled the counts in `scaledCount` to a predefined range of 10 and 50.

We are now ready to start plotting:

In [ ]:
mapped_stats_country_trace = go.Scattergeo(  # we picked the Scattergeo geometry which will load a map
    lon = mapped_stats_country['longitude'],  # we mapped the longitude in the data to the lon aesthetic
    lat = mapped_stats_country['latitude'],
    mode = 'markers',  # we want to have points
    text = [  # the text reflects the information shown in the tooltips...
        "{}<br>Number of Beers: {}<br>Mean Overall Score: {:.2f}".format(
            country,
            count,
            mean
        ) for country, count, mean in zip(
            mapped_stats_country.index.tolist(),  # ...we want it to contain the Country name...
            mapped_stats_country['count'].astype('int'),  # ... the count (transformed to an integer value)...
            mapped_stats_country['mean']  # ...and the average overall score.
        )
    ],
    marker = dict(
        size = mapped_stats_country['scaledCount'],  # the scaledCounts are mapped to the size aesthetic
        color = mapped_stats_country['mean'],  # the average score is mapped to the color aesthetic
        colorscale = [  # we can build our own color scale
            [0.0, 'rgb(165,0,38)'],
            [0.50, 'rgb(254,224,144)'],
            [1.0, 'rgb(143,206,70)']
        ],
        showscale = True
    )
)

traces = [mapped_stats_country_trace]

layout = go.Layout(
    title = 'World Distribution of Beers',
    width = 750,  # this define the size of the plot
    height = 750,
    geo = dict(  # here we define the properties of the map
        showcountries = True,  # whether to show country borders
        showocean = True,  # whether to color the ocean blue
        projection = dict(
            type = 'orthographic',  # the type of projections
            rotation = dict(  # the starting view of the map
                lon = -25,
                lat = 0,
                roll = 0
            )
        )
    )
)

fig = go.Figure(data = traces, layout = layout)

py.offline.iplot(fig)

We can now nicely see the cluster of countries that produce many beers of high quality!

Now it is time to try it yourself. Edit some lines of code, improve the visualisation or change the input data. Here is the link to the [Scattergeo](https://plot.ly/javascript/reference/#scattergeo) geometry which contains all the options that you could pass. Also [here](https://plot.ly/python/scatter-plots-on-maps/) are some fun examples of maps.

In [ ]:
# Use this cell as exploration ground!
# for example: change the markers, change the color, change the map projection, change the geometry...













## Open Questions

The dataset is rich enough to answer many more questions. For instance:

* What are the most popular styles of beer in the UK?
* What are the most successful breweries in the world?
* What beers taste good but lack in appearance?
* ...

Try to answer these questions with some simple summary statistics and engaging plots.

## Take-home Message

* When building plots, we pick a certain geometry that has some aesthetics to which we map the data.
* Summary statistics via `.groupby()` offer a very convenient way to gain insights into the data.

## Sources

Some great materials that I used in the making of this tutorial:

* [IPython Notebook Tutorial](https://plot.ly/python/ipython-notebook-tutorial/)
* [Widgets In Ipython Notebook And Plotly](http://moderndata.plot.ly/widgets-in-ipython-notebook-and-plotly/)
* [EDA and interactive figures with Plotly](http://online.cambridgecoding.com/notebooks/cca_admin/eda-and-interactive-figures-with-plotly)
* [Chris Albon's blog](http://chrisalbon.com/)